# Estimate threshold concentration

1. Calculate total volume of Lake Michigan (units cubic meters)
2. Calculate total annual load entering the lake (units mg)
3. Calculate well mixed concentration (Load/Vol) (units mg/m3 == ug/L)

### Results
- Volume is 4.8987714449E12 m3 (close to what google tells me)
- Annual load is 1.90375E12 mg
- Well mixed concentration is 0.389 mg/m3

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
def get_model_grid():
    """
    docstring
    SA : surface area of each cell (m2)
    depth : depth of each cell (m)
    lon : longitude (deg)
    lat : latitude (deg)
    drF : r cell face separation (m)
    rA : face area at cell (m)
    dlon : diff in lon (deg)
    dlat : diff in lat (deg)
    volCell : volume of surface cells (m3)
    volLake : volume of lake (m3)
    """
    # Read grid file
    gridDir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
    gridFile = gridDir + 'grid_lake_michigan.nc'
    ds = xr.open_dataset(gridFile)

    # Variables
    SA = ds['rA'].values
    depth = ds['Depth'].values
    lon = ds['X'].values
    lat = ds['Y'].values
    drF = ds['drF'].values
    rA = ds['rA'].values
    dlon = np.diff(lon[0:2]) / 2
    dlat = np.diff(lat[0:2]) / 2
    volCell = rA * drF[0]
    vol = SA * depth
    volLake = np.sum(vol.flat)

    # output
    out = {'SA': SA,
           'depth': depth,
           'lon': lon,
           'lat': lat,
           'drF': drF,
           'rA': rA,
           'dlon': dlon,
           'dlat': dlat,
           'volCell': volCell,
           'vol': vol,
           'volLake': volLake}

    return out

# calculate well mixed concentration

In [3]:
### ==========================================================
### Load grid
### ==========================================================
grid = get_model_grid()

### ==========================================================
### Define Variables 
### ==========================================================
vol_columns = grid['depth']*grid['SA']
vol = np.sum(vol_columns.flatten())
#print('Volume is :', vol, 'm3')

### ==========================================================
### 1. Read data
### ==========================================================
ploadDir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
df_dolan = pd.read_csv(ploadDir + 'dolan_pload_1994_2008_MTyr.csv', index_col='river')
df_wrtds = pd.read_csv(ploadDir + 'wrtds_pload_2007_2010_MTyr.csv', index_col='river')

### ==========================================================
### 2. Calculate annual load 
###.   (convert from MT to kg then to mg)
### 1000 kg = 1MT
### 1000000 mg = 1kg
### ==========================================================
annual_load = (np.sum(np.sum(df_wrtds[['pload2007',
                                       'pload2008',
                                       'pload2009',
                                       'pload2010']],axis=0).values)/4)* 1000 * 1000000
#print(annual_load, ' mg')

### ==========================================================
### 2. Calculate the well mixed
###    concentration by dividing by the lake volume
### ==========================================================
conc = annual_load/vol
print(conc, ' mg/m3')

0.388617844578734  mg/m3
